In [1]:
! pip3 install --upgrade emcee corner pytorch-lightning tqdm scipy zuko


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip3 install --upgrade pip


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import time, sys, os
import matplotlib.pyplot as plt

# Simulation-based (likelihood-free) inference
By **Jessie Micallef** ([jessiem@mit.edu](mailto:jessiem@mit.edu))

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jessimic/sbi-tutorial-iaifi/blob/main/Tutorial3_Open_Dataset.ipynb)

# 1. Introduction for Tutorial 3

Let's put together these tools to apply to a real dataset. We will load in a simulation, run SBI, and infer on a different simulation to see the robustness or weakness of SBI. This is the most advanced tutorial, with minimal answers available.

## 1.1 The dataset: 

We will use the [CAMELS Multifield Dataset](https://camels-multifield-dataset.readthedocs.io/en/latest/index.html) (CMD). "CMD is a publicly available collection of hundreds of thousands 2D maps and 3D grids containing different properties of the gas, dark matter, and stars from more than 12,000 different universes". These files are quite large, so we will only download two datasets. The first is for training our SBI model: [Maps_T_IllustrisTNG_LH_z=0.00.npy](https://users.flatironinstitute.org/~fvillaescusa/priv/DEPnzxoWlaTQ6CjrXqsm0vYi8L7Jy/CMD/2D_maps/data/IllustrisTNG/Maps_T_IllustrisTNG_LH_z=0.00.npy). The second will be later to run inference on for comparison: [Maps_T_Astrid_LH_z=0.00.npy](https://users.flatironinstitute.org/~fvillaescusa/priv/DEPnzxoWlaTQ6CjrXqsm0vYi8L7Jy/CMD/2D_maps/data/Astrid/Maps_T_Astrid_LH_z=0.00.npy).

The site recommends uploading a dataset to your local google drive, then mounting it in colab to access your files:

In [ ]:
drive.mount('/content/drive')

## 1.2 Plot one of the images in IllustrisTNG

In [ ]:
splits     = 6   #number of maps per simulation

fmaps2 = '/content/drive/MyDrive/Images_T_IllustrisTNG_LH_z=0.00.npy'
maps  = np.load(fmaps2)
print('Shape of the maps:',maps.shape)

######### ADD PLOTTING SCRIPT HERE ##########

To help with loading, you can choose a small number of images:

In [ ]:
# define the array that will contain the indexes of the maps
indexes = np.zeros(1000*splits, dtype=np.int32)

# do a loop over all maps and choose the ones we want
count = 0
for i in range(15000):
    if i%15 in np.arange(splits):  
      indexes[count] = i
      count += 1
print('Selected %d maps out of 15000'%count)

# save these maps to a new file
maps = maps[indexes]
np.save('maps_T.npy', maps)
del maps

"One of the most obvious applications of CMD is parameter inference. Each 2D map and 3D grid is characterized by 6 numbers: two cosmological parameters describing fundamental properties of the Universe and four astrophysical parameters that quantify the efficiency of astrophysical processes supernova explosions or feedback from black holes." These six parameters are summarized as Omega_m, sigma_8, A_SN1, A_AGN1, A_SN2, A_AGN2. For ease, we will focus on predicting the first two parameters, Omega_m and sigma_8, which are the "two cosmological parameters from the data with the highest accuracy." [[1]](https://camels-multifield-dataset.readthedocs.io/en/latest/inference.html)

## 1.3 Previous models and hints at dataloader

The dataset comes with a model that has previously been trained. We can load the model, rather than trying to retrain, to make a comparison with our SBI. [Benchmark Script in Colab](https://colab.research.google.com/drive/1-BmkA8JSc36O8g9pj7FenD1YSLKqjQR3?usp=sharing) also has hints on how to use dataloader on these files.

# Further reading

- [The frontier of simulation-based inference](https://arxiv.org/abs/1911.01429) (Cranmer, Brehmer, Louppe): Review paper
- [simulation-based-inference.org](http://simulation-based-inference.org/): List of papers and resources
- [awesome-neural-sbi](https://github.com/smsharma/awesome-neural-sbi): List of papers and resources

<img src=./assets/header.png alt= “” width=800>
